## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config3 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database.ipynb/WeatherPy_database.csv")
city_data_df.head()

,City,City.1,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Tura,IN,2022-03-16 16:11:49,25.5198,90.2201,74.84,27,0,3.96,clear sky
1,1,Puerto Ayora,EC,2022-03-16 16:11:49,-0.7393,-90.3518,82.54,83,71,5.99,broken clouds
2,2,Leningradskiy,RU,2022-03-16 16:11:50,69.3833,178.4167,-13.63,95,59,9.26,broken clouds
3,3,Katsuura,JP,2022-03-16 16:11:50,35.1333,140.3000,57.15,83,26,7.45,scattered clouds
4,4,Hilo,US,2022-03-16 16:11:50,19.7297,-155.0900,73.13,94,100,4.00,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City,City.1,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Puerto Ayora,EC,2022-03-16 16:11:49,-0.7393,-90.3518,82.54,83,71,5.99,broken clouds
6,6,Tres Passos,BR,2022-03-16 16:11:51,-27.4556,-53.9319,77.47,66,61,7.92,broken clouds
9,9,San Patricio,US,2022-03-16 16:11:53,28.0170,-97.5169,75.29,44,0,7.83,clear sky
12,12,Saint-Joseph,RE,2022-03-16 16:11:54,-21.3667,55.6167,79.18,87,65,9.33,light rain
21,21,Salalah,OM,2022-03-16 16:11:58,17.0151,54.0924,80.69,69,0,3.44,clear sky
23,23,Georgetown,MY,2022-03-16 16:11:58,5.4112,100.3354,80.53,89,20,3.44,few clouds
27,27,Auki,SB,2022-03-16 16:12:00,-8.7676,160.7034,76.53,87,95,3.71,overcast clouds
29,29,Tiarei,PF,2022-03-16 16:12:01,-17.5333,-149.3333,75.22,71,39,1.59,scattered clouds
31,31,Nouakchott,MR,2022-03-16 16:07:09,18.0858,-15.9785,85.93,37,0,10.94,clear sky
32,32,Papara,BR,2022-03-16 16:12:02,-3.9839,-38.7273,81.55,69,88,5.82,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City           0
City.1         0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# all row are full and good to go for the analysis
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, weather description, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,1,EC,82.54,broken clouds,-0.7393,-90.3518,
6,6,BR,77.47,broken clouds,-27.4556,-53.9319,
9,9,US,75.29,clear sky,28.0170,-97.5169,
12,12,RE,79.18,light rain,-21.3667,55.6167,
21,21,OM,80.69,clear sky,17.0151,54.0924,
23,23,MY,80.53,few clouds,5.4112,100.3354,
27,27,SB,76.53,overcast clouds,-8.7676,160.7034,
29,29,PF,75.22,scattered clouds,-17.5333,-149.3333,
31,31,MR,85.93,clear sky,18.0858,-15.9785,
32,32,BR,81.55,overcast clouds,-3.9839,-38.7273,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
print("Finished searching for hotels")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Finished searching for hotels


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == ""].index)


In [21]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,1,EC,82.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,6,BR,77.47,broken clouds,-27.4556,-53.9319,Hotel Imperial
9,9,US,75.29,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
12,12,RE,79.18,light rain,-21.3667,55.6167,"""Plantation Bed and Breakfast"
21,21,OM,80.69,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
...,...,...,...,...,...,...,...
678,678,ID,76.12,few clouds,-3.9450,122.4989,Hotel Wixel
681,681,MX,80.60,scattered clouds,20.0500,-90.2333,Hotel Mangrove King Fishing
682,682,GF,79.84,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
683,683,MZ,80.55,scattered clouds,-17.8786,36.8883,Hotel Flamingo


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Description<dt/><dd>{Description}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
plt.savefig("Vacation_Search/WeatherPy_vacation_map.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>